# 라이브러리 설치

In [1]:
!pip install -q chromadb sentence-transformers openai easyocr opencv-python

# API Key 환경 설정

In [2]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN') # Hugging Face Token
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY') # Google Custom Search API 키
GOOGLE_CSE_ID = userdata.get('GOOGLE_CSE_ID') # Programmable Search Engine ID
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY') # OpenAI API Key
client_id = userdata.get('NAVER_CLIENT_ID') # Naver Client Key
client_secret = userdata.get('NAVER_CLIENT_SECRET') # Naver Client Secret
gpt_target = "gpt-4.1-mini"

# Base Knowledge 수집용 네이버쇼핑 영양제 상품 검색 코드

In [3]:
import requests
from urllib.parse import quote

def naver_shop_search(query, display=100):
    encoded_query = quote(query)
    url = f"https://openapi.naver.com/v1/search/shop.json?query={encoded_query}&display={display}&start=1&sort=sim&exclude=used:cbshop:rental"

    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        return {"error": response.status_code}

result = naver_shop_search("영양제", 100)

# 추가 정보 수집을 위한 구글 검색 코드

In [4]:
from googleapiclient.discovery import build

def google_search(query, num=10):
    service = build("customsearch", "v1", developerKey=GOOGLE_API_KEY)
    res = service.cse().list(q=query, cx=GOOGLE_CSE_ID, num=num).execute()
    return [item["snippet"] for item in res["items"]]


# Base Knowledge를 Vector DB에 저장하는 코드

In [5]:
# 네이버쇼핑 검색 결과 -> Vector DB (Chroma) 저장
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer

# import cv2
# import easyocr
# import requests
# import numpy as np
# from matplotlib import pyplot as plt
# import logging
# logging.getLogger('easyocr').setLevel(logging.ERROR)


# def image_recognition(image_url):
#   if image_url:
#     response = requests.get(image_url)
#     image_array = np.asarray(bytearray(response.content), dtype=np.uint8)
#     img = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
#     if img is not None: # Add this check
#         reader = easyocr.Reader(['en', 'ko'])
#         results = []
#         try:
#           results = reader.readtext(img)
#         except Exception as e: # Catch specific exceptions if possible, or log the error
#           print(f"Error reading image text for {image_url}: {e}") # Log the error
#         textlist = []
#         for box,text,conf in results:
#           if text:
#             textlist.append(text)
#         return ','.join(textlist)
#     else:
#       print(f"Failed to decode image from URL: {image_url}") # Log decoding failure
#       return ''
#   else:
#     return ''

# 데이터 전처리
def preprocess_items(items):
    processed = []
    count=1
    for item in items:
        data = {
            "id": item['productId'],
            "text": f"{item['title']} {item['brand']} {item['maker']} {item['category3']} {item['category4']}",  # 임베딩용 텍스트
            "metadata": {
                "price": int(item['lprice']),
                "brand": item['brand'],
                "category": f"{item['category1']}>{item['category2']}>{item['category3']}>{item['category4']}",
                "link": item['link'],
                # "imagetext": image_recognition(item['image'])
                "imagelink": (item['image'])
            }
        }
        processed.append(data)
        # print("count : "+ str(count)+ " / " + str(len(items)))
        # count+=1
    return processed

# Chroma DB 초기화
client = chromadb.PersistentClient(path="./naver_shopping_db")
embedding_model = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2", token=HF_TOKEN
)
# 기존 컬렉션이 있으면 삭제 후 재생성
try:
    client.delete_collection("products")
except:
    pass
collection = client.create_collection(
    name="products",
    embedding_function=embedding_model,
    metadata={"hnsw:space": "cosine"}
)

# 데이터 저장
if 'items' in result:
    processed_data = preprocess_items(result['items'])
    collection.add(
        ids=[item['id'] for item in processed_data],
        documents=[item['text'] for item in processed_data],
        metadatas=[item['metadata'] for item in processed_data]
    )
    print("성공적으로", len(processed_data), "개 상품 저장됨")
else:
    print("Error:", result.get('error', '데이터 없음'))

성공적으로 100 개 상품 저장됨


# Vector DB 검색 코드

In [6]:
def vectordb_search(query, n_results=10):
    results = collection.query(
        query_texts=[query],
        n_results=n_results
    )
    output = []
    for idx, (doc, meta) in enumerate(zip(results['documents'][0], results['metadatas'][0]), 1):
        output.append({
            "제품정보": doc,
            "메타데이터": meta
        })
    return output
vectordb_search("비타민")

[{'제품정보': '아임비타 멀티 비타민 이뮨 플러스 7일 샷 종근당 액상 아이엠 비타 이문 종합 종근당  비타민제 멀티비타민',
  '메타데이터': {'category': '식품>건강식품>비타민제>멀티비타민',
   'imagelink': 'https://shopping-phinf.pstatic.net/main_8660875/86608759572.7.jpg',
   'brand': '종근당',
   'price': 11880,
   'link': 'https://smartstore.naver.com/main/products/9064259249'}},
 {'제품정보': '센트룸 실버 포 우먼 종합 멀티 비타민 <b>영양제</b> 112정 코스트코 비타민B12 대용량 센트룸 화이자 비타민제 멀티비타민',
  '메타데이터': {'category': '식품>건강식품>비타민제>멀티비타민',
   'brand': '센트룸',
   'price': 34900,
   'imagelink': 'https://shopping-phinf.pstatic.net/main_8678780/86787803460.8.jpg',
   'link': 'https://smartstore.naver.com/main/products/9243303137'}},
 {'제품정보': '암웨이 더블엑스 리필 종합비타민 미네랄 뉴트리라이트 암웨이  비타민제 멀티비타민',
  '메타데이터': {'price': 67000,
   'brand': '암웨이',
   'category': '식품>건강식품>비타민제>멀티비타민',
   'imagelink': 'https://shopping-phinf.pstatic.net/main_8329310/83293108601.13.jpg',
   'link': 'https://smartstore.naver.com/main/products/5748609307'}},
 {'제품정보': '센트룸 실버 포 맨 종합 멀티 비타민 112정 코스트코 남성 50+ 대용량 <b>영양제</b> 센

# 검색 도구 함수 정의 (Function Calling)

In [7]:
recommend_nutrient = {
    "name": "recommend_nutrient",
    "description": "사용자가 입력한 상태를 분석해 영양소를 추천합니다.",
    "parameters": {
        "type": "object",
        "properties": {
            "nutrient": {"type": "string", "description": "추천하는 영양소 성분"},
            "query": {"type": "string", "description": "사용자 상태 쿼리"
            }
        },
        "required": ["nutrient", "query"]
    }
}

excessive_nutrient = {
    "name": "excessive_nutrient",
    "description": "사용자가 입력한 영양제 섭취 상태를 분석해 일일 권장량을 초과하는 영양소를 찾습니다.",
    "parameters": {
        "type": "object",
        "properties": {
            "nutrient": {"type": "string", "description": "초과하는 영양소 성분"},
            "query": {"type": "string", "description": "계산 수식과 초과한 이유"
            }
        },
        "required": ["nutrient", "query"]
    }
}

vectordb_search_schema = {
  "name": "vectordb_search",
  "description": "추천 영양소로 Vector DB에서 제품 검색",
  "parameters": {
    "type": "object",
    "properties": {
      "query": {"type": "string"},
      "max_results": {"type": "integer", "default": 10}
    },
    "required": ["query"]
  }
}

google_search_function_schema = {
  "name": "google_search",
  "description": "검색된 제품을 Google 검색해서 상세 정보 조회",
  "parameters": {
    "type": "object",
    "properties": {
      "query": {"type": "string"},
      "num_results": {"type": "integer", "default": 10}
    },
    "required": ["query"]
  }
}


# LLM 동작 코드1

In [14]:
from openai import OpenAI
import json
import re

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

def nutrient_too_much_check(symptom, gender, age, pregnancy, mode):
    prompt = f"""
    사용자 정보:
    - 성별: {gender}
    - 나이: {age}
    - 임신 여부: {pregnancy}
    - 원하는 것: {mode}
    - 상태: {symptom}

    현재 사용자가 섭취하고 있는 상태인 각각 영양소의 총 함량과 해당 영양소의 일일 권장량을 비교하여 초과/미달 여부를 계산하세요. 계산 근거를 같이 제시하세요.
    """
    messages = [
        {
            "role": "system",
            "content": "너는 건강 기반의 영양제 조언 전문가야. 전문적으로 답변해줘."
        },
        {"role": "user", "content": prompt}
    ]

    print("현재 messages:", messages)

    res = client.chat.completions.create(
        model=gpt_target,
        messages=messages,
        functions=[excessive_nutrient],
        function_call={"name": "excessive_nutrient"},
        temperature=0.5
    )

    func_name = res.choices[0].message.function_call.name
    print("호출된 함수:", func_name)
    print("함수 인자(raw):", res.choices[0].message.function_call.arguments)

    args = json.loads(res.choices[0].message.function_call.arguments)
    print(args)

    messages.append({
        "role": "assistant",
        "content": None,
        "function_call": {
            "name": func_name,
            "arguments": res.choices[0].message.function_call.arguments
        }
    })

    messages.append({
        "role": "function",
        "name": func_name,
        "content": f"과다 복용 분석 결과: {json.dumps(args, ensure_ascii=False)}"
    })

    # 최종 응답 포맷: 성분 중심 카드 형태
    messages.append({
        "role": "user",
        "content": """
각 성분별로 아래 형식으로 정리해주세요:

┌───────────────────────────────────────────────────────┐
│ 성분명     │ [예: 비타민 D]
├───────────────────────────────────────────────────────┤
│ 섭취량     │ [숫자 및 단위]
├───────────────────────────────────────────────────────┤
│ 권장량     │ [숫자 및 단위]
├───────────────────────────────────────────────────────┤
│ 초과량     │ [±숫자 및 단위]
├───────────────────────────────────────────────────────┤
│ 초과 여부  │ ✅ 초과 섭취 / ❌ 미달 섭취
└───────────────────────────────────────────────────────┘

🛡️ AI Agent의 분석 결과입니다.
복용 중인 약물이나 지병이 있다면 반드시 전문가와 상담하세요.
        """
    })

    final_response = client.chat.completions.create(
        model=gpt_target,
        messages=messages,
        temperature=0.5
    )

    print("\n최종 결과:")
    print(final_response.choices[0].message.content)


def nutrient_recommend_check(symptom, gender, age, pregnancy, mode):
    prompt = f"""
    사용자 정보:
    - 성별: {gender}
    - 나이: {age}
    - 임신 여부: {pregnancy}
    - 원하는 것: {mode}
    - 상태: {symptom}
    """

    if mode == "식습관 분석을 통한 영양제 추천":
        prompt += "\n현재 사용자의 식사 습관 상태를 고려하여 결핍되거나 보조적으로 섭취가 필요한 가장 적절한 영양소를 2가지 추천하세요.\n"
    elif mode == "불편 현상에 따른 영양제 추천":
        prompt += "\n현재 사용자가 불편해 하는 상태를 해결할 수 있는 가장 적절한 영양소를 2가지 추천하세요.\n"

    nutrient_recommend(prompt, symptom, gender, age, pregnancy, mode)


def nutrient_recommend(condition, symptom, gender, age, pregnancy, mode):
    prompt = condition
    messages = [
        {
            "role": "system",
            "content": "너는 건강 기반의 영양제 조언 전문가야. 전문적으로 답변해줘."
        },
        {"role": "user", "content": prompt}
    ]

    print("현재 messages:", messages)

    res = client.chat.completions.create(
        model=gpt_target,
        messages=messages,
        functions=[recommend_nutrient],
        function_call={"name": "recommend_nutrient"},
        temperature=0.5
    )

    func_name = res.choices[0].message.function_call.name
    print("호출된 함수:", func_name)
    print("함수 인자(raw):", res.choices[0].message.function_call.arguments)

    args = json.loads(res.choices[0].message.function_call.arguments)
    nutrient = args["nutrient"]
    print(f"추천 영양소: {nutrient}")

    # Step 2: VectorDB 검색
    search_results = vectordb_search(args["query"])
    messages.append({
        "role": "assistant",
        "content": None,
        "function_call": {
            "name": func_name,
            "arguments": res.choices[0].message.function_call.arguments
        }
    })

    messages.append({
        "role": "function",
        "name": func_name,
        "content": f"VectorDB 검색 결과: {json.dumps(search_results)}"
    })

    messages.append({
        "role": "user",
        "content": "Vector DB에서 검색한 추천하는 영양소를 포함하는 영양제 제품 중 제일 적합한 영양제 제품 2개를 찾으세요"
    })

    res = client.chat.completions.create(
        model=gpt_target,
        messages=messages,
        functions=[vectordb_search_schema],
        function_call={"name": "vectordb_search"},
        temperature=0.5
    )

    func_name = res.choices[0].message.function_call.name
    print("호출된 함수:", func_name)
    print("함수 인자(raw):", res.choices[0].message.function_call.arguments)
    print("메세지: ", res.choices[0].message)

    # Step 3: Google 검색
    args = json.loads(res.choices[0].message.function_call.arguments)
    search_results = google_search(args["query"])

    messages.append({
        "role": "assistant",
        "content": None,
        "function_call": {
            "name": func_name,
            "arguments": res.choices[0].message.function_call.arguments
        }
    })

    messages.append({
        "role": "function",
        "name": func_name,
        "content": f"Google 검색 결과: {json.dumps(search_results)}"
    })

    messages.append({
        "role": "user",
        "content": "찾은 영양제 제품을 Google에서 검색하여 자세한 함량 정보를 수집하세요."
    })

    res = client.chat.completions.create(
        model=gpt_target,
        messages=messages,
        functions=[google_search_function_schema],
        function_call={"name": "google_search"},
        temperature=0.5
    )

    func_name = res.choices[0].message.function_call.name
    print("호출된 함수:", func_name)
    print("함수 인자(raw):", res.choices[0].message.function_call.arguments)
    print("메세지: ", res.choices[0].message)

    # 최종 응답 포맷: 제품 정보 카드 형태
    messages.append({
        "role": "user",
        "content": """
지금까지의 검색 결과를 종합해서 아래 형식으로 보기 좋게 정리해주세요.

┌────────────────────────────────────────────────────┐
│ 제조사     │
├────────────────────────────────────────────────────┤
│ 제품명     │
├────────────────────────────────────────────────────┤
│ 주요 성분  │
├────────────────────────────────────────────────────┤
│ 함량       │
├────────────────────────────────────────────────────┤
│ 링크       │
└────────────────────────────────────────────────────┘

사용자의 안전을 위해 다음 문구를 마지막에 추가하세요:
'AI Agent가 조언하는 내용입니다. 건강에 이상이 있거나 복용 중인 약물이 있다면 전문가 상담을 받으세요.'
        """
    })

    final_response = client.chat.completions.create(
        model=gpt_target,
        messages=messages,
        temperature=0.5
    )

    print("\n최종 결과:")
    print(final_response.choices[0].message.content)


def llm_divider(symptom, gender, age, pregnancy, mode):
    if mode in ["식습관 분석을 통한 영양제 추천", "불편 현상에 따른 영양제 추천"]:
        nutrient_recommend_check(symptom, gender, age, pregnancy, mode)
    elif mode == "영양제 과다 복용 확인":
        nutrient_too_much_check(symptom, gender, age, pregnancy, mode)

### LLM동작코드2

In [12]:
from openai import OpenAI
import json
import re

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

def nutrient_too_much_check(symptom, gender, age, pregnancy, mode):
    prompt = f"""
    사용자 정보:
    - 성별: {gender}
    - 나이: {age}
    - 임신 여부: {pregnancy}
    - 원하는 것: {mode}
    - 상태: {symptom}

    현재 사용자가 섭취하고 있는 상태인 각각 영양소의 총 함량과 해당 영양소의 일일 권장량을 비교하여 초과/미달 여부를 계산하세요. 계산 근거를 같이 제시하세요.
    """
    messages = [
        {
            "role": "system",
            "content": "너는 건강 기반의 영양제 조언 전문가야. 전문적으로 답변해줘."
        },
        {"role": "user", "content": prompt}
    ]

    res = client.chat.completions.create(
        model=gpt_target,
        messages=messages,
        functions=[excessive_nutrient],
        function_call={"name": "excessive_nutrient"},
        temperature=0.5
    )

    func_name = res.choices[0].message.function_call.name
    args = json.loads(res.choices[0].message.function_call.arguments)

    messages.append({
        "role": "assistant",
        "content": None,
        "function_call": {
            "name": func_name,
            "arguments": res.choices[0].message.function_call.arguments
        }
    })

    messages.append({
        "role": "function",
        "name": func_name,
        "content": f"과다 복용 분석 결과: {json.dumps(args, ensure_ascii=False)}"
    })

    messages.append({
        "role": "user",
        "content": """
각 성분별로 아래 형식으로 정리해주세요:

┌───────────────────────────────────────────────────────┐
│ 성분명     │ [예: 비타민 D]
├───────────────────────────────────────────────────────┤
│ 섭취량     │ [숫자 및 단위]
├───────────────────────────────────────────────────────┤
│ 권장량     │ [숫자 및 단위]
├───────────────────────────────────────────────────────┤
│ 초과량     │ [±숫자 및 단위]
├───────────────────────────────────────────────────────┤
│ 초과 여부  │ ✅ 초과 섭취 / ❌ 미달 섭취
└───────────────────────────────────────────────────────┘

🛡️ AI Agent의 분석 결과입니다.
복용 중인 약물이나 지병이 있다면 반드시 전문가와 상담하세요.
        """
    })

    final_response = client.chat.completions.create(
        model=gpt_target,
        messages=messages,
        temperature=0.5
    )

    print(final_response.choices[0].message.content)


def nutrient_recommend_check(symptom, gender, age, pregnancy, mode):
    prompt = f"""
    사용자 정보:
    - 성별: {gender}
    - 나이: {age}
    - 임신 여부: {pregnancy}
    - 원하는 것: {mode}
    - 상태: {symptom}
    """

    if mode == "식습관 분석을 통한 영양제 추천":
        prompt += "\n현재 사용자의 식사 습관 상태를 고려하여 결핍되거나 보조적으로 섭취가 필요한 가장 적절한 영양소를 2가지 추천하세요.\n"
    elif mode == "불편 현상에 따른 영양제 추천":
        prompt += "\n현재 사용자가 불편해 하는 상태를 해결할 수 있는 가장 적절한 영양소를 2가지 추천하세요.\n"

    nutrient_recommend(prompt, symptom, gender, age, pregnancy, mode)


def nutrient_recommend(condition, symptom, gender, age, pregnancy, mode):
    prompt = condition
    messages = [
        {
            "role": "system",
            "content": "너는 건강 기반의 영양제 조언 전문가야. 전문적으로 답변해줘."
        },
        {"role": "user", "content": prompt}
    ]

    res = client.chat.completions.create(
        model=gpt_target,
        messages=messages,
        functions=[recommend_nutrient],
        function_call={"name": "recommend_nutrient"},
        temperature=0.5
    )

    func_name = res.choices[0].message.function_call.name
    args = json.loads(res.choices[0].message.function_call.arguments)
    nutrient = args["nutrient"]

    # Step 2: VectorDB 검색
    search_results = vectordb_search(args["query"])
    messages.append({
        "role": "assistant",
        "content": None,
        "function_call": {
            "name": func_name,
            "arguments": res.choices[0].message.function_call.arguments
        }
    })

    messages.append({
        "role": "function",
        "name": func_name,
        "content": f"VectorDB 검색 결과: {json.dumps(search_results)}"
    })

    messages.append({
        "role": "user",
        "content": "Vector DB에서 검색한 추천하는 영양소를 포함하는 영양제 제품 중 제일 적합한 영양제 제품 2개를 찾으세요"
    })

    res = client.chat.completions.create(
        model=gpt_target,
        messages=messages,
        functions=[vectordb_search_schema],
        function_call={"name": "vectordb_search"},
        temperature=0.5
    )

    func_name = res.choices[0].message.function_call.name
    args = json.loads(res.choices[0].message.function_call.arguments)

    # Step 3: Google 검색
    top_products = [result["제품정보"] for result in search_results[:2]]
    google_queries = [f"{product} 영양제" for product in top_products]
    google_results = []
    for query in google_queries:
        google_results.extend(google_search(query))

    messages.append({
        "role": "assistant",
        "content": None,
        "function_call": {
            "name": "google_search",
            "arguments": json.dumps({"query": " / ".join(google_queries), "num_results": 10}, ensure_ascii=False)
        }
    })

    messages.append({
        "role": "function",
        "name": "google_search",
        "content": f"Google 검색 결과: {json.dumps(google_results, ensure_ascii=False)}"
    })

    # ✅ 변경된 출력 요청 메시지 (표 2개로 출력)
    messages.append({
        "role": "user",
        "content": """
지금까지의 검색 결과를 종합해서 아래 형식의 표를 **각 제품마다 개별적으로** 작성해주세요.

┌────────────────────────────────────────────────────┐
│ 제조사     │
├────────────────────────────────────────────────────┤
│ 제품명     │
├────────────────────────────────────────────────────┤
│ 주요 성분  │
├────────────────────────────────────────────────────┤
│ 함량       │
├────────────────────────────────────────────────────┤
│ 링크       │
└────────────────────────────────────────────────────┘

📌 제품이 2개라면 위 표를 **2번 반복해서 출력**해주세요.

🛡️ AI Agent가 조언하는 내용입니다.
건강에 이상이 있거나 복용 중인 약물이 있다면 전문가 상담을 받으세요.
        """
    })

    final_response = client.chat.completions.create(
        model=gpt_target,
        messages=messages,
        temperature=0.5
    )

    print(final_response.choices[0].message.content)


def llm_divider(symptom, gender, age, pregnancy, mode):
    if mode in ["식습관 분석을 통한 영양제 추천", "불편 현상에 따른 영양제 추천"]:
        nutrient_recommend_check(symptom, gender, age, pregnancy, mode)
    elif mode == "영양제 과다 복용 확인":
        nutrient_too_much_check(symptom, gender, age, pregnancy, mode)


# User Interface 실행

In [15]:
import gradio as gr
import io
import sys

def nutrient_advisor(symptom, gender, age, pregnancy, mode):
    old_stdout = sys.stdout
    sys.stdout = mystdout = io.StringIO()
    llm_divider(symptom, gender, age, pregnancy, mode)
    sys.stdout = old_stdout
    return mystdout.getvalue()

with gr.Blocks() as demo:
    gr.Markdown("# 영양제 추천/과다 복용 확인")
    gender = gr.Radio(["남성", "여성"], label="성별")
    age = gr.Number(value=30, label="만 나이")
    pregnancy = gr.Radio(["해당 없음", "임신 중", "임신 가능성"], label="임신 여부")
    mode = gr.Radio(
        ["식습관 분석을 통한 영양제 추천", "불편 현상에 따른 영양제 추천", "영양제 과다 복용 확인"],
        label="메뉴"
    )
    symptom = gr.Textbox(label="증상 또는 복용 제품 입력", placeholder="증상 또는 복용 제품 입력")
    submit = gr.Button("제출")
    output = gr.Textbox(label="결과", lines=15)

    submit.click(
        nutrient_advisor,
        inputs=[symptom, gender, age, pregnancy, mode],
        outputs=output
    )

demo.launch(share=True, debug=False)
